# ICAPS24 SkDecide Tutorial: solving problems (possibly imported from Gym) with Reinforcement Learning and Cartesian Genetic Programming

This tutorial shows how to load a domain in scikit-decide and try to solve it with techniques from different communities:

*   [Reinforcement Learning](https://en.wikipedia.org/wiki/Reinforcement_learning) (RL)
*   [Cartesian Genetic Programming](https://en.wikipedia.org/wiki/Cartesian_genetic_programming) (CGP)

## Prerequisites

Install scikit-decide:

In [ ]:
!pip install "scikit-decide[all]"

Install renderlab to render Gymnasium environments in Google Colab:

In [ ]:
!pip install renderlab

## Loading a domain

Once a problem is formalized as a scikit-decide domain, it can be tackled by any compatible solver. Domains can be created from scratch or imported from various formats. Here we demonstrate how to import an environment from [Gymnasium](https://gymnasium.farama.org) (the new official fork of OpenAI Gym, a standard API often used in RL communities), like [Cart Pole](https://gymnasium.farama.org/environments/classic_control/cart_pole):

In [1]:
import gymnasium as gym
from renderlab import RenderFrame
from skdecide.hub.domain.gym import GymDomain

# Select a Gymnasium environment
ENV_NAME = "CartPole-v1"

# Create a domain factory, a callable returning a skdecide domain (used by solvers)
def domain_factory():

    # Create a Gymnasium environment
    env = gym.make(ENV_NAME, render_mode = "rgb_array")

    # Wrap it with RenderFrame to record/play episode videos (works in Colab)
    env = RenderFrame(env, "./render")

    # Return a skdecide domain from a Gymnasium environment
    return GymDomain(env)

# In simple cases, domain_factory can be created in one line:
# domain_factory = lambda: GymDomain(gym.make(ENV_NAME))

The rollout utility provides a quick way to run episodes by taking random actions (or a solver policy as shown later) in the domain:

In [2]:
from skdecide.utils import rollout

# Instantiate one domain (used for rollouts)
domain = domain_factory()

# Do a random rollout of the domain (random actions are taken when no solver is specified)
rollout(domain, num_episodes=1, max_steps=1000) # try adding outcome_formatter=None or verbose=False for less/no printing
domain.unwrapped().play() # watch last episode in video by calling play() on the underlying Gymnasium environment (works in Colab)

2024-05-30 09:21:55,838 | skdecide.utils | DEBUG | Logger is in verbose mode: all debug messages will be there for you to enjoy （〜^∇^ )〜
2024-05-30 09:21:55,864 | skdecide.utils | DEBUG | Episode 1 started with following observation:
2024-05-30 09:21:55,867 | skdecide.utils | DEBUG | [ 0.01505552 -0.0485096   0.01376329  0.03647679]
2024-05-30 09:21:55,877 | skdecide.utils | DEBUG | Action: 0
2024-05-30 09:21:55,887 | skdecide.utils | DEBUG | Result: EnvironmentOutcome(observation=array([ 0.01408533, -0.24382618,  0.01449282,  0.3334702 ], dtype=float32), value=Value(reward=1.0, cost=-1.0), termination=False, info={})
2024-05-30 09:21:55,896 | skdecide.utils | DEBUG | Action: 1
2024-05-30 09:21:55,907 | skdecide.utils | DEBUG | Result: EnvironmentOutcome(observation=array([ 0.00920881, -0.04891348,  0.02116222,  0.04539258], dtype=float32), value=Value(reward=1.0, cost=-1.0), termination=False, info={})
2024-05-30 09:21:55,916 | skdecide.utils | DEBUG | Action: 0
2024-05-30 09:21:55,92

Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


## Solving the domain

One of the key benefits of scikit-decide is its ability to connect the same domain definition to many different solvers from various communities. To demonstrate this versatility, we show how to solve the domain loaded above with both Reinforcement Learning and Cartesian Genetic Programming:

### With Reinforcement Learning (RL)

Scikit-decide provides wrappers for several RL solvers, such as [RLlib](https://docs.ray.io/en/latest/rllib/index.html) and [Stable-Baselines3](https://stable-baselines3.readthedocs.io). We use the latter in this example:

In [3]:
from stable_baselines3 import PPO
from skdecide.hub.solver.stable_baselines import StableBaseline

# Check domain compatibility with StableBaseline RL solver (good practice)
assert StableBaseline.check_domain(domain)

# Instantiate solver with parameters of choice (e.g. type of algo/neural net, learning steps...)
solver = StableBaseline(
    domain_factory,
    algo_class=PPO,
    baselines_policy="MlpPolicy",
    learn_config={"total_timesteps": 10000},
    verbose=1
)

# Solve with RL
solver.solve()

# Save solution
solver.save("saved_solution")

  logger.deprecation(



Using cpu device
-----------------------------
| time/              |      |
|    fps             | 121  |
|    iterations      | 1    |
|    time_elapsed    | 16   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 120         |
|    iterations           | 2           |
|    time_elapsed         | 33          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008190245 |
|    clip_fraction        | 0.0952      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0112     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.32        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0148     |
|    value_loss           | 51          |
-----------------------------------------
-----------------

Now we can run episodes with rollout using the latest solver policy:

In [4]:
# Visualize solution (pass solver to rollout to use its policy)
rollout(domain, solver, num_episodes=1, max_steps=1000, verbose=False)
domain.unwrapped().play()

Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



t:  86%|████████▋ | 83/96 [00:00<00:00, 212.64it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./render/1717061038.634458.mp4, 720000 bytes wanted but 0 bytes read,at frame 95/96, at time 3.17/3.17 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

                                                             

Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


It is always possible to reload a saved solution (especially useful in a new Python session) and possibly continue learning from there. By running this cell a couple of times, you should see increasingly better solutions:

In [5]:
# Optional: reload solution (required if reloading in a new Python session)
solver.load("saved_solution")

# Continue learning
solver.solve()

# Save updated solution
solver.save("saved_solution")

# Visualize updated solution
rollout(domain, solver, num_episodes=1, max_steps=1000, verbose=False)
domain.unwrapped().play()

  logger.deprecation(



-----------------------------
| time/              |      |
|    fps             | 137  |
|    iterations      | 1    |
|    time_elapsed    | 14   |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 130        |
|    iterations           | 2          |
|    time_elapsed         | 31         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00618995 |
|    clip_fraction        | 0.026      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.594     |
|    explained_variance   | 0.667      |
|    learning_rate        | 0.0003     |
|    loss                 | 8.99       |
|    n_updates            | 60         |
|    policy_gradient_loss | -0.00621   |
|    value_loss           | 44.8       |
----------------------------------------
------------------------------------------
| time/  

Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


After using a solver, it is good practice to do a cleanup as shown below (not critical here, but sometimes useful for C++ parallel solvers in scikit-decide). Note that this is automatically done if you use the solver within a `with` statement, which will be shown in the CGP sub-section below as an alternative.

In [6]:
# Clean up solver after use (good practice)
solver._cleanup()

### With Cartesian Genetic Programming (CGP)

In [7]:
from skdecide.hub.solver.cgp import CGP

# Check domain compatibility with CGP solver (good practice)
assert CGP.check_domain(domain)

# Instantiate solver with parameters of choice (using "with" syntax to avoid manual clean up)
with CGP(domain_factory, folder_name="TEMP_CGP", n_it=100) as solver:

    # Solve with CGP
    solver.solve()

    # Visualize solution
    rollout(domain, solver, num_episodes=1, max_steps=1000, verbose=False)
    domain.unwrapped().play()

  x = (x - inmin) / (inmax - inmin)

  return self.randrange(a, b+1)



[Discrete(2)]
[Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)]
[  6   1   1   9   1   3   9   0   2   1   4   4   1   3   4   8   3   8
  19   6   4  18   0   7   7   0   7  19   3   2  11   1   4   1   7  11
   9  12  13  12  10   0  18   2  10   8   4   5  17   3   5  16   0  14
  16  16  16   6   7   2  12   6  11  14  17  18  14   8  16  10   8  11
  19  13   1  20   7  13   4  14   8  12  14   1  12   0   7  18  20  12
   8  32   8   4  16  19   8   3   9  20   2   8  10  28  19  16  37  15
   9   7  21  11   3   4  10  22  35  16  28  37  11  41  37   2  22  20
  15  42  43  18  28  45  12  36  38   1  31  29  20  25  47  13  16  37
   8  19  14   5   1   6  11  23  27  13  27  47  20  11  21  18  12   9
  17  14  14  17   1  50   5  10  26   7  43  50  17  54  37   7  24  39
   0  13  34   1  12  61   2  28  54  16  24  48  14  19   3   0  14  56
  17  11  24   1  51  57   3  38  48   3

Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


In this example, you may find that RL often finds better solutions than CGP (although this depends on the solver parameters and the random seed). Note however that this is highly problem-dependent: try re-running this notebook after setting `ENV_NAME = "MountainCarContinuous-v0"` at the beginning and you may find opposite results. That shows the power of having a wide catalog of solvers to find the best solution for each specific problem!